# Overview
---

The goal of this workshop is to gain familiarity with the computational resources provided by Compute Canada. The workshop will cover aspects of logging in, transfering data, initializing environments, requesting CPU / GPU resources, and relinquishing jobs when you are done using them. To motivate the use of these resources, a deep learning model written in Python will be trained on both CPU and GPU, and the relative performances will be evaluated. 

Much of the information found in this document is taken from [here](https://www.sharcnet.ca/help/index.php/Knowledge_Base), and this [webinar](https://www.youtube.com/watch?v=IiAbxPZ3BHo&feature=youtu.be)


## Workshop Requirements
---

* Compute Canada account
* (WINDOWS) [Putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html) for connecting to SHARCNet
* (WINDOWS) [Psftp](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html) for transfering files between SHARCNet
* (OPTIONAL, WINDOWS) [xming](http://www.straightrunning.com/XmingNotes/) for viewing graphical windows 

## Compute Canada Nodes
---

* [Cedar](https://docs.computecanada.ca/wiki/Cedar): cedar.computecanada.ca 
* [Graham](https://docs.computecanada.ca/wiki/Graham): graham.computecanada.ca
* [Niagara](niagara.computecanada.ca): niagra.computecanada.ca


# 1. SHARCNet
---

__SHARCNet__ stands for _Shared Hierarchical Academic Research Computing Network_. It was established in 2000, and involves 18 Universities and Colleges across Southern, Central, and Northern Ontario. The main office is at the University of Western Ontario.

SHARCNet is free to use for academic researchers. To log in to SHARCNet, you need to establish a connection using the secure shell protocol (SSH). 

## 1.1. Connecting from Linux
---

If connecting from linux, you can connect by entering the following from the command line: 
 
```ssh graham.sharcnet.ca``` 

and you will be prompted to input your username and password. Alternatively, you can also initialize the connection with your username by adding it as a prefix

```ssh username@graham.sharcnet.ca```

which will allow you to log in by only entering your password on the subsequent screen. 

## 1.2. Connecting from Windows
---

Download the program [Putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html), and enter &lt;cluster_name&gt;.sharcnet.ca in the "Host Name (or IP address) field. You shouldn't need to change any other sections

<img src="images/graham_sharcnet.PNG" width="400">


# 2. Major Feature: Login Nodes vs. Development Nodes
---

SHARCNet is a different environment then what you may be used to. Here, you submit a _job_ to a scheduler, which will queue your request for processing until a resource is free. 

When you connect to SHARCNet, you connect to a _login_ node that serves as a sort-of centralized hub. Login nodes are the only nodes that have internet access, and are useful for tasks such as:

* Submitting jobs to the scheduler
* Transfering data to / from the SHARCNet ecosystem
* Initializing code environments
* Coding / processing very small projects

_Development nodes_ on the other hand, are the main workhorses that are designed to handle heavy code processing. 

## 2.1. Interactive Nodes vs. Non-Interactive Nodes
---

When you submit a job to the scheduler, you give it constraints such as the _maximum running time_ and _how much memory is needed_, and the job scheduler will find an appropriate time for your program to run on _non-interactive nodes_. If your program is found to have bugs in it (and it crashes during its alloted execution time), your remaining time will be forfeit and you will be tasked with fixing the code and submitting it for execution again. 

Interactive nodes on the other hand, can be used to quickly prototype code on a particular resource. You can request either CPUs or GPUs for a small amount of time, do some interactive programming, and when you're done relinquish the node and submit the job to a non-interactive node. 

# 3. Major Feature: The Job Scheduler
---

Information from [here](https://www.sharcnet.ca/help/index.php/SLURM) and webinar [here](https://www.youtube.com/watch?v=dfw027j284Q)

Since SHARCNet is shared among a large number of researchers, there is no guarantee that resources will always be available for use. Instead, there is a scheduler that chooses when code can run and depends on factors such as priority (e.g. and how much of the resource you've used before), and the amount of resources requested (e.g. memory, many nodes, etc ...). This scheduler is called __SLURM__, and stands for: Simple Linux Utility for Resource Management. 

An overview of the job scheduling policies can be found [here](https://docs.computecanada.ca/wiki/Job_scheduling_policies).

#  4. Major Feature / Consideration: Data Storage Policies
---

Note: Table copied from [here](https://docs.computecanada.ca/wiki/Storage_and_file_management)

|   Filesystem  |           Default Quota          |            Lustre-based?           |  Backed up? |                  Purged?                |  Available by Default? |  Mounted on Compute Nodes? |
|:-------------:|:--------------------------------:|:----------------------------------:|:-----------:|:---------------------------------------:|:----------------------:|----------------------------|
| Home Space    | 50 GB and 500K files per user[1] | Yes for Cedar, No for Graham (NFS) | Yes         | No                                      | Yes                    | Yes                        |
| Scratch Space | 20 TB and 1M files per user[2]   | Yes                                | No          | Files older than 60 days are purged.[3] | Yes                    | Yes                        |
| Project Space | 1 TB and 500k files per group[4] | Yes                                | Yes         | No                                      | Yes                    | Yes                        |

## 4.1. Notes
---

* /home is accessed by all users, and should not be used to frequently open / load / read from when submitting jobs as it may introduce additionally latency that affects a large number of users
* /scratch is fast access, but files will only exist for 60 days before they get purged. This means that if your program writes any files to the /scratch file system and you want to keep these files, you should move them once done. 

# 5. Consideration: File Transfer
---

There are many different ways to transfer data between your compute and the SHARCNet resources. If you are not comfortable using the command line to transfer data, SHARCNet encourages the use of software known as [Globus](https://www.sharcnet.ca/help/index.php/Globus). This is a portal that operates in a manner similar to "drag and drop" that should be familiar with most users. 

There are also other methods for transferring data such as:

* (LINUX) [scp](https://linux.die.net/man/1/scp) which stands for _secure copy_
* (LINUX) [rsync](https://linux.die.net/man/1/rsync) which is an efficient tool for copying files
* (WINDOWS) [psftp](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html) 

We will discuss some of these later in the workshop.